# Analyse MARC (https://marc.ifremer.fr) data 

In [1]:
import fsspec
import intake
import xarray as xr

catalogue="/home/datawork-taos-s/intranet/kerchunk/ref-marc.yaml"
catalogue = "https://data-taos.ifremer.fr/kerchunk/ref-marc.yaml"

In [2]:
def list_param(cat, param):
    return cat.metadata["parameters"][param]["allowed"]

def allowed_param(cat):
    return cat.metadata["parameters"]

def allowed_years(region):
    fs = fsspec.filesystem("https") if ('https' in catalogue) else fsspec.filesystem("file")
    files = fs.glob(catalogue.replace(".yaml", "/") + region + "/*.json.zst")
    years = [
        f.replace(".json.zst", "").replace(
            catalogue.replace(".yaml", "/") + region + "/", ""
        )
        for f in files
    ]
    years.sort()
    return years

def year_concat(cat, source, region):
    concat = [
        cat[source](region=region, year=i).to_dask() for i in allowed_years(region)
    ]

    return xr.concat(
        concat, dim="time", coords="minimal", data_vars="minimal", compat="override"
    )

Detail of the functions:
-----------------------------------------------------------------------------------------------  
    list_param(cat, param)    
>- Takes 2 arguments: cat = An opened catalog, param = The name of a parameter.
>- Returns a ptyhon list containing all the parameters.

    allowed_param(cat)    
>- Takes 1 argument: cat = An opened catalog
>- Returns a python dict containing the allowed values of each parameters and their description

    Allowed_years(region)    
>- Takes 1 argument: region = A region of data.
>- Returns a pythton list containing all the year available in kerchunk catalogue.  

    year_concat(cat, source, region)    
>- Takes 3 arguments: cat = An opened catalog, source = the source from this catalog, region = the region where the data comes from
>- Returns: A dataset concatenated over the all available in kerchunk. 


# Open intake catalgue.  

In [3]:
cat = intake.open_catalog(catalogue)

## Inspect intake catalogue

In [4]:
regions = list_param(cat, "region")

In [5]:
regions

['f1_e2500_agrif/MARC_F1-MARS3D-ADOUR',
 'f1_e2500_agrif/MARC_F1-MARS3D-ARMOR',
 'f1_e2500_agrif/MARC_F1-MARS3D-FINIS',
 'f1_e2500_agrif/MARC_F1-MARS3D-GIRONDE',
 'f1_e2500_agrif/MARC_F1-MARS3D-LOIRE',
 'f1_e2500_agrif/MARC_F1-MARS3D-MANGAE2500-AGRIF',
 'f1_e2500_agrif/MARC_F1-MARS3D-MORBIHAN',
 'f1_e2500_agrif/MARC_F1-MARS3D-PDC',
 'f1_e2500_agrif/MARC_F1-MARS3D-SEINE',
 'l1_v9/MARC_L1-MARS2D-AQUI250',
 'l1_v9/MARC_L1-MARS2D-ATLNE2000',
 'l1_v9/MARC_L1-MARS2D-FINIS250',
 'l1_v9/MARC_L1-MARS2D-MANE250',
 'l1_v9/MARC_L1-MARS2D-MANGA700',
 'l1_v9/MARC_L1-MARS2D-MANW250',
 'l1_v9/MARC_L1-MARS2D-SUDBZH250',
 'b1',
 'b1_v10',
 'b1_v10r2019',
 'f1_e2500',
 'f1_e4000',
 'f2_1200',
 'f2_1200_sn',
 'f2_1200_v10',
 'l1_brest',
 'l2_v10',
 'ww3/ANTILLES-3MIN',
 'ww3/ANTILLES-UG',
 'ww3/ATNE-10M',
 'ww3/FINIS-200M',
 'ww3/GLOBAL-30MIN',
 'ww3/MED-6MIN',
 'ww3/MENOR-2MIN',
 'ww3/MENOR-UG',
 'ww3/NORGAS-2MIN',
 'ww3/NORGAS-UG',
 'ww3/NORGAS-UG_V3',
 'ww3/REUNION-UG']

## Define a region, inspect the catalogue to know which year the datas are availableand, then load the data

In [6]:
region_local = "f1_e2500_agrif/MARC_F1-MARS3D-SEINE"

In [7]:
allowed_years(region_local)

['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

In [8]:
region_global = "f1_e2500_agrif/MARC_F1-MARS3D-MANGAE2500-AGRIF"

In [9]:
allowed_years(region_global)

['2017', '2018']

In [13]:
year='2018'
time=['2018-06-23T00:00:00.000008', '2018-06-23T12:00:00.000000']
time='2018-06-23'

In [14]:
ds_local = cat.marc(region=region_local , year=year).to_dask().sel(time=time)
ds_global = cat.marc(region=region_global, year=year).to_dask().sel(time=time)

## Plot data

In [16]:
%%time
da_local = ds_local.TEMP.unify_chunks()
da_global = ds_global.TEMP.unify_chunks()



CPU times: user 1.6 ms, sys: 137 µs, total: 1.74 ms
Wall time: 2.45 ms


In [19]:
#import geoviews.feature as gf
import hvplot.xarray

(
da_local.hvplot.quadmesh(
    x="longitude", y="latitude", rasterize=True, geo=True, cmap="BuPu", coastline=True
)

 ) 
#* gf.land.opts(scale="10m") * gf.coastline.opts(scale="10m")

/Users/todaka/micromamba/envs/pangeo-fish_new/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/todaka/micromamba/envs/pangeo-fish_new/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/todaka/micromamba/envs/pangeo-fish_new/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/todaka/micromamba/envs/pangeo-fish_new/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/todaka/micromamba/envs/pangeo-fish_new/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/todaka/micromamba/envs/pangeo-fish_new/lib/python3.10/site-packages/shapely/con

/Users/todaka/micromamba/envs/pangeo-fish_new/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/todaka/micromamba/envs/pangeo-fish_new/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(


:DynamicMap   [time,level]
   :Overlay
      .Image.I     :Image   [longitude,latitude]   (TEMP)
      .Coastline.I :Feature   [Longitude,Latitude]

In [ ]:
da_global.hvplot.quadmesh(
    x="longitude", y="latitude", rasterize=True, geo=True, cmap="BuPu", coastline=True
)